# Check library versions of acquisition

In [1]:
import time
import numpy as np
from Include.acq import CACodeDetectionResult, detect_ca_sat, detect_ca_sat_gpu
from Include.readAcqData import readAcqData
from init_settings import init_settings
from Include.plotAcqSearch import plotAcqSearch
import cupy as cp
from cupyx.scipy.fft import fft as cp_fft, ifft as cp_ifft
from cupyx.scipy.fftpack import get_fft_plan  # plan reuse
from Include.makeCaTable import make_ca_table


In [2]:
settings = init_settings()
print (f'Data File: {settings.fileName}')
longdata = readAcqData(settings, framing = False)

Data File: /mnt/e/gnss_data/L1_IF20KHz_FS18MHz/L1_IF20KHz_FS18MHz.bin


In [3]:
gpu_results = detect_ca_sat_gpu(17, longdata, settings, n_coherent = 5, n_noncoherent = 10)
print (gpu_results)
gpu_matrix = gpu_results.detector
gpu_peak = gpu_results.peak_location()
print (f"GPU computed Peak Location: \n{gpu_peak}")

CACodeDetectionResult(success=True, prn=17, n_coherent=5, n_noncoherent=10, detector=array([[0.34786037, 0.32188493, 0.32905498, ..., 0.36630815, 0.364623  ,
        0.36780262],
       [0.1493465 , 0.13313822, 0.14796858, ..., 0.13520293, 0.14868288,
        0.13778646],
       [0.1160528 , 0.10830019, 0.11982413, ..., 0.1224874 , 0.1237236 ,
        0.11941185],
       ...,
       [0.30124694, 0.27618322, 0.26068285, ..., 0.27003184, 0.34115958,
        0.3008919 ],
       [0.26360518, 0.25563526, 0.25026366, ..., 0.28417063, 0.30336985,
        0.27497613],
       [0.42250434, 0.4275116 , 0.42316163, ..., 0.42354137, 0.4090296 ,
        0.41870427]], shape=(141, 180000), dtype=float32), freq_table=array([-7000., -6900., -6800., -6700., -6600., -6500., -6400., -6300.,
       -6200., -6100., -6000., -5900., -5800., -5700., -5600., -5500.,
       -5400., -5300., -5200., -5100., -5000., -4900., -4800., -4700.,
       -4600., -4500., -4400., -4300., -4200., -4100., -4000., -3900.,
      

In [4]:
cpu_results = detect_ca_sat(17, longdata, settings, n_coherent = 5, n_noncoherent = 10)
print (cpu_results)
cpu_matrix = cpu_results.detector
cpu_peak = cpu_results.peak_location()
print (f"CPU computed Peak Location: \n{gpu_peak}")

CACodeDetectionResult(success=True, prn=17, n_coherent=5, n_noncoherent=10, detector=array([[0.34786057, 0.32188505, 0.32905513, ..., 0.3663084 , 0.36462322,
        0.36780286],
       [0.14934653, 0.13313824, 0.1479686 , ..., 0.13520299, 0.14868294,
        0.13778648],
       [0.11605284, 0.10830026, 0.11982418, ..., 0.12248753, 0.12372369,
        0.11941196],
       ...,
       [0.30124706, 0.27618322, 0.26068282, ..., 0.27003175, 0.34115973,
        0.30089206],
       [0.26360524, 0.25563538, 0.25026375, ..., 0.28417084, 0.3033701 ,
        0.2749763 ],
       [0.42250463, 0.42751175, 0.42316192, ..., 0.42354184, 0.4090301 ,
        0.41870478]], shape=(141, 180000), dtype=float32), freq_table=array([-7000., -6900., -6800., -6700., -6600., -6500., -6400., -6300.,
       -6200., -6100., -6000., -5900., -5800., -5700., -5600., -5500.,
       -5400., -5300., -5200., -5100., -5000., -4900., -4800., -4700.,
       -4600., -4500., -4400., -4300., -4200., -4100., -4000., -3900.,
      

In [10]:
def detect_ca_sat_gpu(
    prn: int,
    long_data: np.ndarray,
    settings,
    show_status: bool = False,
    n_coherent: int | None = None,
    n_noncoherent: int | None = None,
) -> CACodeDetectionResult:
    #
    #
    t0 = time.time()
    t_sample = 1.0 / settings.samplingFreq
    low_f  = -float(settings.acqSearchBand)
    high_f =  float(settings.acqSearchBand)
    if n_coherent    is None: n_coherent    = int(settings.acqCoherentInt)
    if n_noncoherent is None: n_noncoherent = int(settings.acqNonCohTime)

    samples_per_code = int(round(settings.codeLength * settings.samplingFreq / settings.codeFreqBasis))
    N = 2 * samples_per_code * n_coherent   # samples per coherent vector; IFFT length / columns
    B = n_noncoherent
    total_samples = samples_per_code * (2 * n_coherent * n_noncoherent)

    if len(long_data) < total_samples:
        return CACodeDetectionResult(False, prn, n_coherent, n_noncoherent, None, None, time.time()-t0, "Insufficient Data")

    # ---- Move + preprocess on GPU
    x = cp.asarray(long_data, dtype=cp.complex64)
    rl = x.real; im = x.imag
    rl -= rl.mean(); im -= im.mean()
    scale = cp.maximum(rl.max(), im.max())
    x = ((0.5 / scale) * (rl + 1j*im)).astype(cp.complex64, copy=False)

    IF = float(getattr(settings, "IF", 0.0))
    if IF != 0.0:
        w = cp.float32(2.0 * cp.pi * t_sample * IF)
        n = cp.arange(x.size, dtype=cp.float32)
        x *= cp.exp(-1j * w * n).astype(cp.complex64, copy=False)

    # ---- Compute Doppler bins
    freqs = cp.fft.fftfreq(N, t_sample)
    mask = (freqs >= low_f) & (freqs <= high_f)
    F = int(mask.sum().get())   #get() method pulls the sum off the GPU and back to CPU
    initial_shift = (F - 1) // 2
    freq_table_gpu = cp.roll(freqs, initial_shift)[:F].copy()
    
    # ---- PRN template: extend TD & compute FD
    # Move the PRN onto the GPU, then make a new vector that is
    # copy nCoherent copies and an equal length of zeros
    pn = cp.asarray(make_ca_table(prn, settings), dtype=cp.complex64)
    prn_td = cp.concatenate((cp.tile(pn, n_coherent),
                             cp.zeros(samples_per_code * n_coherent, dtype=cp.complex64)))
    prn_td = cp.ascontiguousarray(prn_td)
    with get_fft_plan(prn_td, value_type='C2C'):
        prn_fd_conj = cp.conj(cp_fft(prn_td))  # (N,)

    # ---- Reshape into B blocks (e.g. B = nNoncoherent) and FFT each (plan once)
    data = x[:total_samples].reshape(B, N)
    data = cp.ascontiguousarray(data)
    with get_fft_plan(data, axes=(1,), value_type='C2C'):
        X_fd = cp_fft(data, axis=1)  # (B, N), contiguous

    # ---- Precompute T_fd for bin 0 and a cheap per-bin update
    # Bin i corresponds to rolling by s = initial_shift - i (mod N).
    # We'll maintain a rolling view of prn_fd_conj using indices 
    base_idx = cp.arange(N, dtype=cp.int32)
    # start with s0 = initial_shift
    idx = (base_idx + initial_shift) % N
    # preallocate work buffers to avoid re-allocation costs
    conv_fd = cp.empty((B, N), dtype=cp.complex64)
    ifft_out = cp.empty((B, N), dtype=cp.complex64)
    detector_gpu = cp.zeros((F, N), dtype=cp.float32)
    #
    # We normally think about looping over the non-coherent integrations
    # and adding together matrixes several matrixes each of dimension [doppler][sample].
    # It turns out that is not the best approach on a GPU -it is fastest 
    # to loop over the doppler frequencies, and 
    # at the end of the loop perform all the no-coherent integrations 
    # for that doppler frequency.  This is because
    # n_noncoherent < n_frequencies, this minimizes the gather time
    # at the end of the loop.
    #
    # One cuFFT plan for the (B, N) inverse transforms, reused across all bins
    with get_fft_plan(ifft_out, axes=(1,), value_type='C2C'):
        for i in range(F):
            # Gather shifted template spectrum for this Doppler bin
            T_fd = prn_fd_conj[idx]          # (N,)
            # Broadcast multiply (B, N) * (N,) -> (B, N)
            cp.multiply(X_fd, T_fd, out=conv_fd)
            # Batched IFFT over N for all B blocks
            ifft_out[...] = cp_ifft(conv_fd, axis=1)
            # Power & accumulate over B
            # detector row i: sum |ifft_out|^2 across B
            detector_gpu[i] = cp.sum((ifft_out.real * ifft_out.real + ifft_out.imag * ifft_out.imag), axis = 0, dtype = cp.float32)
            #power = cp.abs(ifft_out)**2
            #detector_gpu[i] = power.sum(axis=0).astype(cp.float32)
            # Update index for next bin: shift right by 1 (equivalent to roll by -1 later)
            idx = (idx - 1) % N

    # Normalize & move back to CPU
    detector_gpu *= (1.0 / (B * samples_per_code))
    detector = cp.asnumpy(detector_gpu)
    freq_table = cp.asnumpy(freq_table_gpu)
    return CACodeDetectionResult(True, prn, n_coherent, n_noncoherent, detector, freq_table, time.time()-t0)